In [3]:
# things we need for NLP
import nltk
nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

# things we need for Tensorflow
import numpy as np
import tflearn
import tensorflow as tf
import random

[nltk_data] Downloading package punkt to C:\Users\tigani
[nltk_data]     ismail\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [6]:
# import our chat-bot intents file
import json
with open('intents.json') as json_data:
    intents = json.load(json_data)

In [7]:
words = []
classes = []
documents = []
ignore_words = ['?']
# loop through each sentence in our intents patterns
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = nltk.word_tokenize(pattern)
        # add to our words list
        words.extend(w)
        # add to documents in our corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# stem and lower each word and remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# remove duplicates
classes = sorted(list(set(classes)))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

66 documents
14 classes ['bank-branch', 'duration', 'goodbye', 'greeting', 'howyoureply', 'opentoday', 'payments', 'price', 'proc', 'req', 'thanks', 'time', 'whoareyou', 'whomadeyou']
109 unique stemmed words ['ابدا', 'ابوابكم', 'اتبرمجت', 'اتصنعت', 'اجي', 'اجيب', 'اخدها', 'ادفع', 'اراك', 'استخراج', 'اسمك', 'اشيل', 'اشيلها', 'اقدم', 'الاجراءات', 'الاصال', 'الاوراق', 'الايصال', 'البتتعاملو', 'البرمجك', 'البنك', 'البنوك', 'التقديم', 'الحاجات', 'الدفع', 'السلام', 'السلامة', 'الشغل', 'الشهادة', 'الصممك', 'الطورك', 'العايزنها', 'العمل', 'العملك', 'الفرع', 'الفروع', 'الكلية', 'الليلة', 'المتاحة', 'المطلوبات', 'المطلوبة', 'الوقت', 'اليوم', 'اليوم؟', 'ان', 'انت', 'اي', 'ايام', 'بتاخد', 'بترد', 'بشنو', 'بكرة', 'بنك', 'تستغرق', 'تكلف', 'حالك', 'رائع', 'زول', 'ساعات', 'ستفتحون', 'سعر', 'سعيد', 'سلام', 'سموك', 'شخص', 'شغال', 'شغالين', 'شكرا', 'شنو', 'طريقة', 'عايز', 'عليكم', 'فرع', 'فروع', 'في', 'قروش', 'قصرت', 'كم', 'كنت', 'كيف', 'لاحقا', 'لقد', 'لك', 'للشهادة', 'للشهادة؟', 'لو', 'ما', 'ماهي', 'م

In [1]:
# create our training data
training = []
output = []
# create an empty array for our output
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stem each word
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create our bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)

# create train and test lists
train_x = list(training[:,0])
train_y = list(training[:,1])


NameError: name 'classes' is not defined

In [ ]:
# reset underlying graph data
tf.reset_default_graph()
# Build neural network
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

#Define model and setup tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')
# Start training (apply gradient descent algorithm)
model.fit(train_x, train_y, n_epoch=1000, batch_size=8, show_metric=True)
model.save('model.tflearn')

Training Step: 2329  | total loss: 0.56680 | time: 0.100s
| Adam | epoch: 259 | loss: 0.56680 - acc: 0.8546 -- iter: 56/66


In [13]:
import pickle
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "training_data", "wb" ) )
